In [ ]:
import numpy as np 
from scipy.interpolate import RegularGridInterpolator

def integrate_particles(positions, concentrations, velocity_field, X, Y, dt, num_steps, L):
    #positions is of the form np.ones((6, 2)), concentrations of the form np.ones((6, 1))
    num_particles = len(positions)
    positions_over_time = np.zeros((num_steps+1, num_particles, 2))  #positions_over_time[0] gives positions at t=0, right shape checked!
    concentrations_over_time = np.zeros((num_steps+1, num_particles, 1))
    positions_over_time[0] = positions
    #print(positions_over_time) still correct shape!
    concentrations_over_time[0] = concentrations

    u, v = get_matrix_u(velocity_field), get_matrix_v(velocity_field)
    polynomial_u, polynomial_v = interpolate(u), interpolate(v)
    #print(positions_over_time) still correct shape!
    t_span = (0, num_steps * dt)
    t_eval = [i * dt for i in range(num_steps+1)]
    #y0 = [ [positions[i][0], positions[i][1]] for i in range(num_particles)]
    t = [i*dt for i in range(num_steps+1)]
    #y0 = positions

    du, dv = get_matrix_du(u), get_matrix_dv(v)
    polynomial_du, polynomial_dv = interpolate(du), interpolate(dv) 
    new_vector_rk4(fun, t, polynomial_u, polynomial_v, positions_over_time, polynomial_du, polynomial_dv, concentrations_over_time, num_particles)

    return positions_over_time, concentrations_over_time


def get_matrix_u(velocity_field):
    umatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        umatrix[i] = np.gradient(velocity_field[i], axis=1) * -1
    return umatrix

def get_matrix_v(velocity_field):
    vmatrix = velocity_field
    nt, nx, ny = velocity_field.shape
    for i in range(nt):
        vmatrix[i] = np.gradient(velocity_field[i], axis=0) 
    return vmatrix

def get_matrix_du(umatrix):
    dumatrix = umatrix
    nt, nx, ny = umatrix.shape
    for i in range(nt):
        dumatrix[i] = np.gradient(umatrix[i], axis=0) 
    return dumatrix

def get_matrix_dv(umatrix):
    dumatrix = umatrix
    nt, nx, ny = umatrix.shape
    for i in range(nt):
        dumatrix[i] = np.gradient(umatrix[i], axis=1) 
    return dumatrix

def interpolate(umatrix):

    t = umatrix.shape[0]
    time = [i for i in range(t)]

    interpolator = RegularGridInterpolator((time, X, Y), umatrix, method = "cubic")

    return interpolator

def new_vector_rk4(f, t, polynomial_u, polynomial_v, positions_over_time, polynomial_du, polynomial_dv, concentrations_over_time, num_particles):
    #y0 will be of shape num_particles(N) x 2, same for all k values 
    #t = [dt, 2dt, 3dt,...]
    #f takes a scalar t[i] and a N x 2 array
    n = len(t)
    y = np.concatenate((positions_over_time, concentrations_over_time), axis=2)

    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        k2 = f(t[i] + 0.5*h, y[i] + 0.5*h*k1, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        k3 = f(t[i] + 0.5*h, y[i] + 0.5*h*k2, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        k4 = f(t[i+1], y[i] + h*k3, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    return y

def fun(t, y, polynomial_u, polynomial_v, polynomial_du, polynomial_dv, num_particles):
    #this function should give us the change in position and change in velocity

    xlist = [(i+L)% (2*L) -L for i in y[:, 0]] 
    ylist = [(i+L)% (2*L) -L for i in y[:, 1]] 
    conclist = [(i+L)% (2*L) -L for i in y[:, 2]] 
    timecoords = t*np.ones(num_particles)
    #now all coords are in row format [1,2,3,etc...]
    #now create the txy coords
    coords = [[x, y, z] for x, y, z in zip(timecoords, xlist, ylist)]
    #we want to generate dx/dt dy/dt and dc/dt and the concatenate them
    dxdt = polynomial_u(coords) 
    dydt = polynomial_v(coords)
    #make columns
    dxdt = np.array(dxdt).reshape(-1, 1)
    dydt = np.array(dydt).reshape(-1, 1)

    dudx = polynomial_du(coords)
    dvdy = polynomial_dv(coords)
    column = np.array(dvdy+dudx).reshape(-1, 1)
    c = np.array(conclist).reshape(-1, 1)
    dcdt = -1*c*(column)
    #now all derivatives are columns so we concatenate them

    #testing
    #print(dxdt)
    #print(dydt)
    #testing

    result = np.concatenate((dxdt, dydt), axis=1)
    result = np.concatenate((result, dcdt), axis=1)
    return result